In [1]:
import os
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from tqdm import tqdm

In [2]:
np.random.seed(42)
torch.manual_seed(42)

# Study population

In [3]:
data_dir = "../datafiles/chexpert/"
input_csv = "chexpert.sample_128.test.csv"

# data_dir = '../datafiles/mimic/'
# input_csv = 'mimic.sample.test.csv'

output_suffix = "resample"

sample_size_factor = 3

#### Preprocessing

In [4]:
df = pd.read_csv(data_dir + input_csv).drop(columns="Unnamed: 0")


def bin_age(age):
    if age <= 20:
        return 0
    if age <= 30:
        return 1
    if age <= 40:
        return 2
    if age <= 50:
        return 3
    if age <= 60:
        return 4
    if age <= 70:
        return 5
    if age <= 80:
        return 6
    return 7


df["age_bin"] = df["age"].apply(bin_age)

#### Resample df to have equal proportion of Asian, Black, White

In [5]:
n_samples = len(df) * sample_size_factor
w_race = (1 / df.value_counts("race_label", normalize=True)).to_dict()
ids = list(
    torch.utils.data.WeightedRandomSampler(
        df["race_label"].apply(lambda x: w_race[x]).values, n_samples, replacement=True
    )
)
df_balanced_race = df.iloc[ids]
df_balanced_race.value_counts("race_label", normalize=True)

race_label
0    0.336515
2    0.332165
1    0.331320
dtype: float64

#### Within each (race)-subgroup  ensure equal prevalence. Within each (race, disease)-subsubgroup ensure equal age distribution.

In [10]:
df_balanced_race.race_label.unique()

array([2, 0, 1])

In [6]:
subgroups = [
    df_balanced_race.loc[df_balanced_race["race_label"] == i]
    for i in df_balanced_race.race_label.unique()
]
overall_proportion_disease = df.value_counts("disease_label", normalize=True)
print(overall_proportion_disease)
disease_labels = df.disease_label.unique()
overall_proportion_age_per_disease_label = [
    df.loc[df.disease_label == d].value_counts("age_bin", normalize=True)
    for d in disease_labels
]

for i, subgroup in enumerate(subgroups):
    # 1. Ensure constant disease prevalence in each subgroup
    n_samples = len(subgroup)
    observed_proportion = subgroup.value_counts("disease_label", normalize=True)
    print(observed_proportion)
    w = (overall_proportion_disease / observed_proportion).to_dict()
    ids = list(
        torch.utils.data.WeightedRandomSampler(
            subgroup["disease_label"].apply(lambda x: w[x]).values,
            n_samples,
            replacement=True,
        )
    )
    subgroups[i] = subgroup.iloc[ids]

    # 2. Further ensure constant age repartition across groups
    subsubgroups = [
        subgroups[i].loc[subgroups[i].disease_label == d] for d in disease_labels
    ]
    for j, subsubgroup in enumerate(subsubgroups):
        n_samples = len(subsubgroup)
        observed_proportion = subsubgroup.value_counts("age_bin", normalize=True)
        w = (
            overall_proportion_age_per_disease_label[j] / observed_proportion
        ).to_dict()
        ids = list(
            torch.utils.data.WeightedRandomSampler(
                subsubgroup["age_bin"].apply(lambda x: w[x]).values,
                n_samples,
                replacement=True,
            )
        )
        subsubgroups[j] = subsubgroup.iloc[ids]

    subgroups[i] = pd.concat(subsubgroups)

balanced_df = pd.concat(subgroups)

disease_label
2    0.507688
1    0.405596
0    0.086715
dtype: float64
disease_label
2    0.560962
1    0.328820
0    0.110219
dtype: float64
disease_label
2    0.505971
1    0.410620
0    0.083409
dtype: float64
disease_label
2    0.473756
1    0.439080
0    0.087164
dtype: float64


#### Check balanced dataset

In [ ]:
balanced_df.groupby(["race"]).size()

In [ ]:
balanced_df.groupby(["race", "disease_label"]).size()

In [ ]:
balanced_df.groupby(["race", "disease_label", "age_bin"]).size()

#### Save balanced dataset

In [ ]:
balanced_df.to_csv(data_dir + input_csv.replace("sample", output_suffix))

## Plots

In [ ]:
df_cxr = pd.read_csv(data_dir + input_csv.replace("sample", output_suffix))

In [ ]:
df_cxr.loc[df_cxr["disease"] == "Pleural Effusion", "disease"] = "Pleur. Eff."
df_cxr.loc[df_cxr["disease"] == "No Finding", "disease"] = "No Find."

In [ ]:
df_subgroup = df_cxr[df_cxr["race_label"] == 0]
prev_0 = len(df_subgroup[df_subgroup["disease_label"] == 0]) / len(df_subgroup)
prev_1 = len(df_subgroup[df_subgroup["disease_label"] == 1]) / len(df_subgroup)
prev_2 = len(df_subgroup[df_subgroup["disease_label"] == 2]) / len(df_subgroup)
print("")
print("Prevalence - White")
print("No-finding:\t\t{:.2f}".format(prev_0))
print("Pleural effusion:\t{:.2f}".format(prev_1))
print("Other:\t\t\t{:.2f}".format(prev_2))

df_subgroup = df_cxr[df_cxr["race_label"] == 1]
prev_0 = len(df_subgroup[df_subgroup["disease_label"] == 0]) / len(df_subgroup)
prev_1 = len(df_subgroup[df_subgroup["disease_label"] == 1]) / len(df_subgroup)
prev_2 = len(df_subgroup[df_subgroup["disease_label"] == 2]) / len(df_subgroup)
print("")
print("Prevalence - Asian")
print("No-finding:\t\t{:.2f}".format(prev_0))
print("Pleural effusion:\t{:.2f}".format(prev_1))
print("Other:\t\t\t{:.2f}".format(prev_2))

df_subgroup = df_cxr[df_cxr["race_label"] == 2]
prev_0 = len(df_subgroup[df_subgroup["disease_label"] == 0]) / len(df_subgroup)
prev_1 = len(df_subgroup[df_subgroup["disease_label"] == 1]) / len(df_subgroup)
prev_2 = len(df_subgroup[df_subgroup["disease_label"] == 2]) / len(df_subgroup)
print("")
print("Prevalence - Black")
print("No-finding:\t\t{:.2f}".format(prev_0))
print("Pleural effusion:\t{:.2f}".format(prev_1))
print("Other:\t\t\t{:.2f}".format(prev_2))

In [ ]:
fontsize = 16
plt.rc("xtick", labelsize=fontsize)
plt.rc("ytick", labelsize=fontsize)

fig = sns.catplot(
    x="sex",
    y="age",
    hue="race",
    order=["Male", "Female"],
    hue_order=["White", "Asian", "Black"],
    kind="box",
    data=df_cxr,
)
fig._legend.remove()
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Age (years)", fontsize=fontsize)
plt.ylim([0, 100])
plt.legend(fontsize=fontsize, loc="lower center", ncol=3)
plt.show()
# fig.savefig("CheXpert-resample-age-sex-race.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="disease",
    y="age",
    hue="race",
    order=["Other", "Pleur. Eff.", "No Find."],
    hue_order=["White", "Asian", "Black"],
    kind="box",
    data=df_cxr,
)
fig._legend.remove()
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Age (years)", fontsize=fontsize)
plt.ylim([0, 100])
plt.legend(fontsize=fontsize, loc="lower center", ncol=3)
plt.show()
# fig.savefig("CheXpert-resample-age-pathology-race.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="disease",
    hue="sex",
    order=["Other", "Pleur. Eff.", "No Find."],
    hue_order=["Male", "Female"],
    data=df_cxr,
    kind="count",
)
fig._legend.remove()
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.legend(fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-sex-pathology.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="disease",
    hue="race",
    order=["Other", "Pleur. Eff.", "No Find."],
    hue_order=["White", "Asian", "Black"],
    data=df_cxr,
    kind="count",
)
fig._legend.remove()
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.legend(fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-race-pathology.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="disease",
    y="age",
    order=["Other", "Pleur. Eff.", "No Find."],
    kind="box",
    data=df_cxr,
)
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Age (years)", fontsize=fontsize)
plt.ylim([0, 100])
plt.show()
# fig.savefig("CheXpert-resample-age-pathology.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(x="sex", y="age", order=["Male", "Female"], kind="box", data=df_cxr)
sns.stripplot(
    x="sex",
    y="age",
    color="k",
    order=["Male", "Female"],
    alpha=0.01,
    data=df_cxr,
    ax=fig.ax,
)
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Age (years)", fontsize=fontsize)
plt.ylim([0, 100])
plt.show()
# fig.savefig("CheXpert-resample-age-sex.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="race", y="age", order=["White", "Asian", "Black"], kind="box", data=df_cxr
)
sns.stripplot(
    x="race",
    y="age",
    color="k",
    order=["White", "Asian", "Black"],
    alpha=0.01,
    data=df_cxr,
    ax=fig.ax,
)
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Age (years)", fontsize=fontsize)
plt.ylim([0, 100])
plt.show()
# fig.savefig("CheXpert-resample-age-race.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="race", order=["White", "Asian", "Black"], data=df_cxr, kind="count"
)
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-race.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(x="sex", order=["Male", "Female"], data=df_cxr, kind="count")
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-sex.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="disease", order=["Other", "Pleur. Eff.", "No Find."], data=df_cxr, kind="count"
)
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-pathology.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="race",
    hue="sex",
    order=["White", "Asian", "Black"],
    hue_order=["Male", "Female"],
    data=df_cxr,
    kind="count",
)
fig._legend.remove()
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.legend(fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-sex-race.png", bbox_inches='tight', dpi=300)

In [ ]:
fig = sns.catplot(
    x="sex",
    hue="race",
    order=["Male", "Female"],
    hue_order=["White", "Asian", "Black"],
    data=df_cxr,
    kind="count",
)
fig._legend.remove()
plt.xlabel("", fontsize=fontsize)
plt.ylabel("Count", fontsize=fontsize)
plt.legend(fontsize=fontsize)
plt.show()
# fig.savefig("CheXpert-resample-race-sex.png", bbox_inches='tight', dpi=300)